# RSA Encryption

## Generating the keys

To generate a private key/public key pair, we start by picking two (large) prime numbers, $p$ and $q$:

In [1]:
p = 2    # both p and q should be large, but for illustration
q = 7    # we'll just pick small, easy-to-work-with numbers

With the product $pq=N$, we define $\phi (N)$ to be the count of numbers coprime with $N$:

In [2]:
N = p * q

def phi(p: int, q: int):
    """The count of numbers coprime with p*q,
    given that p and q are prime numbers."""
    return (p - 1) * (q - 1)

Note that this works because, for a prime number $p$, $\phi (p)=(p-1)$, and given *two* prime numbers $p$ and $q$, $\phi (pq)=\phi (p) \times \phi (q)$.

Now we need to choose a number $e$ which will be used in our public key. The public key will ultimately take the form $(e,N)$.

$e$ has these requirements:
1. $1<e<\phi (N)$
2. $e$ is coprime with $N$ and $\phi (N)$

We'll use the `gcd` function (greatest common *divisor*) from the `math` library to check for coprimes.

In [3]:
from math import gcd

In [4]:
def choose_e(p: int, q: int):
    _phi = phi(p, q)
    N = p * q
    options = []
    for i in range(2, _phi):
        if gcd(N, i) == 1 and gcd(_phi, i) == 1:
            options.append(i)
    if len(options) > 1:
        print('More than one option!', options)
    return options[0]

In [5]:
choose_e(p, q)

5

For the private key, which will ultimately have the form $(d,N)$, we need to choose a number $d$ such that $d\times e \space(mod \space \phi (N))=1$:

In [6]:
def choose_d(e: int, _phi: int):
    d = 1
    while (d * e) % _phi != 1:
        d += 1
    # we could just return the first d we find, but there's a whole family
    # such that (d*e)%phi == 1; returning a larger d makes it harder to guess
    return d + _phi

In [7]:
choose_d(choose_e(p, q), phi(p, q))

11

Now let's write the whole operation as one function:

In [8]:
from random import choice, randint

def rsa_keys(p: int, q: int):
    N = p * q
    phi = (p - 1) * (q - 1)
    e_options = []
    for i in range(2, phi):
        if gcd(N, i) == 1 and gcd(phi, i) == 1:
            e_options.append(i)
    e = choice(e_options)
    d = 1
    while (d * e) % phi != 1:
        d += 1
    d += randint(2, 255) * phi
    
    return {'pk': (e, N), 'sk': (d, N)}

## Encryption/Decryption

Let's see an example using our private key/public key pair, given by our `rsa_keys` function.

In [9]:
keys = rsa_keys(p, q)
keys

{'pk': (5, 14), 'sk': (371, 14)}

Let's separate our private & public keys, and create a message to be encrypted.

*Note:* since $p$ and $q$ are small, we can't encrypt large numbers here without losing information to collisions, although in general this won't be an issue. 

For now, let's encode the message '$abc$' as $[1, 2, 3]$; *i.e.* $a\rightarrow 1$, $b\rightarrow 2$, and $c\rightarrow 3$.

In [10]:
pk = keys['pk'] # public key
sk = keys['sk'] # secret key
message = [ord(c) - 97 + 1 for c in 'abcdef']
message         # just a list of numbers now

[1, 2, 3, 4, 5, 6]

For each number in our message, we will raise it to the power $e$ in our public key, $(e,N)$, then take the answer $mod \space N$.

In [11]:
e, N = pk
encrypted_message = [(c**e) % N for c in message]
encrypted_message

[1, 4, 5, 2, 3, 6]

Represented with letters, this becomes:

In [12]:
''.join([chr(c + 97 - 1) for c in encrypted_message])

'adebcf'

In [13]:
d, N = sk
decrypted_message = [(c ** d) % N for c in encrypted_message]
decrypted_message

[1, 2, 3, 4, 5, 6]

We've successfully retrieved the original message:

In [14]:
''.join([chr(c + 97 - 1) for c in decrypted_message])

'abcdef'